# Install dependencies

In [8]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /private/var/folders/_3/b70qb3856jv2v4tk1htvvt1w0000gn/T/pip-req-build-mms3c4yp
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /private/var/folders/_3/b70qb3856jv2v4tk1htvvt1w0000gn/T/pip-req-build-mms3c4yp
^C
ERROR: Operation cancelled by user


In [9]:
!pip install qwen-vl-utils

In [10]:
!pip install torchvision

In [11]:
!pip install accelerate

In [1]:
import torch
import os
import json
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from qwen_vl_utils import process_vision_info

In [4]:
if torch.backends.mps.is_available():
    device = "mps"
elif torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(device)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-tgz8rzep
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-tgz8rzep

  Resolved https://github.com/huggingface/transformers to commit 8bd2b1e8c23234cd607ca8d63f53c1edfea27462
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu.


No drones detected.
(140, 410, 170, 430)
(295,248,338,298)
(445, 382, 475, 408)
(245,200,285,220)
(430, 315, 480, 340)
(458,62,531,92)
(470,375,510,405)
(508, 25, 560, 94)
(800,462,840,492)
(438,498,482,518)
(390, 340, 520, 420)
(588,208,612,232)
No drones detected.
No drone detected.
(834,231,924,319)
(670, 361, 740, 415)
(685,309,730,336)
(250, 400, 320, 458)
(712,92,805,164)
(15, 312, 75, 382)
(48, 178, 121, 256)
(110, 230, 193, 303)
(228,552,299,606)
(640,416,719,476)
(690,425,760,480)
(666,351,733,399)
(944,258,1000,314)
(0.03, 0.58, 0.09, 0.63)
(735,171,798,225)
(585,128,643,184)
(250, 400, 350, 500)
(700,85,810,165)
(535,341,610,415)
(214,453,328,533)
(320, 300, 460, 420)
(600,550,690,620)
(610,480,660,517)
(528,373,580,413)
(580,220,683,328)
(645,678,754,750)
(416, 133, 480, 183)
(348,231,441,309)
(325, 148, 397, 166)
(380, 370, 470, 450)
(298,35,434,165)
(380, 140, 524, 260)
(130, 478, 213, 558)
(220, 190, 330, 270)
(128, 494, 196, 556)
(541,398,608,453)
(572,598,637,652)
(250

OutOfMemoryError: CUDA out of memory. Tried to allocate 3.45 GiB. GPU 0 has a total capacity of 22.19 GiB of which 1.94 GiB is free. Process 1385340 has 20.25 GiB memory in use. Of the allocated memory 19.25 GiB is allocated by PyTorch, and 698.40 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", torch_dtype="auto", device_map="auto"
)

processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

In [ ]:
def process_images(data_directory, prompt="Detect drones in the image. If a drone is detected, return only the bounding box coordinates in the format (x, y, w, h) where (x, y) is the top-left corner of the bounding box and w and h are the width and height, no other text or information, only the coordinates"):
    results_dict = {}
    quantity = len(os.listdir(data_directory))

    for filename in os.listdir(data_directory):
        if filename.endswith('.jpg'):

            image_path = os.path.join(data_directory, filename)
            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "image", "image": f"file://{image_path}"},
                        {"type": "text", "text": prompt},
                    ],
                }
            ]



            text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
            image_inputs, video_inputs = process_vision_info(messages)
            inputs = processor(
                text=[text],
                images=image_inputs,
                videos=video_inputs,
                padding=True,
                return_tensors="pt",
            )
            inputs = inputs.to(device)


            generated_ids = model.generate(**inputs, max_new_tokens=128)
            generated_ids_trimmed = [
                out_ids[len(in_ids):] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
            ]
            output_text = processor.batch_decode(
                generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
            )[0]

            print(f"Pending to process {quantity -1}")

            results_dict[filename] = {
                "Inference": output_text
            }

    output_path = os.path.join(data_directory, 'vlm-drones/output/inference_results.json')
    with open(output_path, 'w') as json_file:
        json.dump(results_dict, json_file, indent=4)

    print(f"Results saved to {output_path}")

In [ ]:
process_images('vlm-drones/images/')